# Baseline Classifier

In this notebook, I will create two baseline classifiers. One random and one intelligent that will make predictions based off the peak of the sample in the ranges I found in 02_DataExploration.ipynb. 

The peaks for each analyte are at:
- Copper - [660, 720]
- Cadmium - [530, 580]
- Lead - [580, 620]
- Seawater - N/A (it's a flat baseline)

## Random Classifier

Let's see how the model will perform if it guesses the classes 0-3 at random.

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import KFold
from functools import partial

from scripts.data_processing import get_class_label_to_int_mapping

CLASS_LABEL_TO_INT_MAPPING = get_class_label_to_int_mapping()

sns.set()

DATA_DIR = Path('data/')

# Read in dataframes - good to keep them separate to make plotting easier
cadmium = pd.read_csv(DATA_DIR / 'cadmium.csv', index_col=0)
copper = pd.read_csv(DATA_DIR / 'copper.csv', index_col=0)
lead = pd.read_csv(DATA_DIR / 'lead.csv', index_col=0)
seawater = pd.read_csv(DATA_DIR / 'seawater.csv', index_col=0)

analytes = pd.read_csv(DATA_DIR / 'all_data.csv', index_col=0)

In [3]:
class RandomClassifier:
    """Returns random predictions of ints in the range 0-3.
    """

    def __init__(self, seed=0):
        np.random.seed(seed)
        self.seed = seed

    def fit(self, X, y=None):
        return self

    def predict(self, X, y=None):
        # Return array len(X) containing ints 0-3 randomly chosen
        return np.random.randint(0, 4, len(X))


class MaxPeakClassifier:
    """Returns predictions based off of the maximum peak value of the sample. 
    """

    def fit(self, X, y=None):
        """
        Blank method to enable use with sklearn API.
        """
        return self

    def predict(self, X, y=None):
        """Predict classes 0-3 for X based off of the max peak value.

        From EDA, we know that the peaks for each analyte occur in the
        following index ranges:
        - Cd - (530, 580)
        - Cu - (660, 720)
        - Pb - (580, 620)

        This method checks the ranges and predicts the analyte with the 
        highest value. However, if the average value of each peak is less 
        than 2.5, it predicts seawater (this value was found through EDA 
        too).

        Parameters
        ----------
        X : np.ndarray or pd.Series or pd.DataFrame
            The object containing voltage series for each analyte. 
            Should be of shape (n_samples, 1002) since each series has len
            1002. Minimum length needed is 720 as that is the final peak
            index position that is checked.

            Note: you can also input a single series with shape (1002,).
        y : None, optional
            Placeholder for y to maintain compatibility with sklearn, by default None

        Returns
        -------
        np.ndarray
            1D array containing predictions for each sample as ints. 
            The mapping is:
                Cd - 0
                Cu - 1
                Pb - 2
                Sw - 3
        """
        # Turn Series and DataFrames into NumPy arrays
        if not isinstance(X, np.ndarray):
            X = X.values
        if X.ndim == 1:
            # Reshape single sample 1D inputs to 2D
            X = X.reshape(1, -1)
        # Define peak value ranges
        cu_peak_start = 660
        cu_peak_end = 720

        cd_peak_start = 530
        cd_peak_end = 580

        pb_peak_start = 580
        pb_peak_end = 620

        preds = []
        # Iterate over the rows (i.e. samples) in X
        for sample in X:
            # Get peak values from sample
            cu_peak_value = sample[cu_peak_start:cu_peak_end].max()
            cd_peak_value = sample[cd_peak_start:cd_peak_end].max()
            pb_peak_value = sample[pb_peak_start:pb_peak_end].max()

            peak_values = {
                'Cd': cd_peak_value,
                'Cu': cu_peak_value,
                'Pb': pb_peak_value
                }

            avergae_value = np.mean(list(peak_values.values()))

            # On average, seawater max values are less than 2.5, found by inspection
            if avergae_value <= 2.5:
                pred = CLASS_LABEL_TO_INT_MAPPING['Sw']
                preds.append(pred)
            else:
                analyte_with_biggest_peak = max(peak_values, key=peak_values.get)
                pred = CLASS_LABEL_TO_INT_MAPPING[analyte_with_biggest_peak]
                preds.append(pred)
        return np.array(preds)


def print_scores(model, X, y, average='micro'):
    """Print accuracy, precision, recall and f1 scores for a given
    fit model.

    Parameters
    ----------
    model : A fitted model with a predict() method
        A model that has already been trained on data and is ready
        to make predictions
    X : np.ndarray, pd.Series, pd.DataFrame
        Input array containing samples with shape (n_samples, n_features)
    y : np.ndarray or pd.Series
        True predictions
    average : str, optional
        Determines the averaging performed on the data. Required for 
        multiclass classification. Options are: 'micro' (default),
        ‘micro’, ‘macro’, ‘samples’, ‘weighted’.
    """
    preds = model.predict(X)
    
    scores = {
        'accuracy': accuracy_score, 
        'f1': partial(f1_score, average='micro'), 
        'precision': partial(precision_score, average='micro'),
        'recall': partial(recall_score, average='micro')
    }

    for name, score_func in scores.items():
        if name != 'accuracy':
            print(f'{name}_{average} - {score_func(y, preds):.4f}')
        else:
            print(f'{name} - {score_func(y, preds):.4f}')

In [4]:
X = analytes.drop('label', axis=1)
y = analytes.loc[:, 'label']

random_clf = RandomClassifier()
print('RANDOM CLASSIFIER SCORES')
print_scores(random_clf, X, y)
print()

max_peak_clf = MaxPeakClassifier()
print('MAX PEAK CLASSIFIER SCORES')
print_scores(max_peak_clf, X, y)

RANDOM CLASSIFIER SCORES
accuracy - 0.2571
f1_micro - 0.2571
precision_micro - 0.2571
recall_micro - 0.2571

MAX PEAK CLASSIFIER SCORES
accuracy - 0.8629
f1_micro - 0.8629
precision_micro - 0.8629
recall_micro - 0.8629


We see that the scores are the same for accuracy, f1, precision and recall which seems odd to me. Why is this the case? Am I doing something wrong? Defo want to check this with Waylon. 

Let's also run a KFold and see if it makes a difference. I know I have made these a bit unfair since I have used all the data to make my models... eek should I go back and change that?

In [5]:
def print_kfold_scores(model, X, y, n_splits=5, **kwargs):
    """Runs KFold validation n_splits times and prints the accuracy,
    f1, precision and recall scores for each fold.

    Parameters
    ----------
    model : Model with fit and predict methods
        The model to use for training and predicting
    X : np.ndarray, pd.Series, pd.DataFrame
        Input array containing samples with shape (n_samples, 1002)
    y : np.ndarray or pd.Series
        Output array
    n_splits : int
        The number of folds to create, defaults to 5
    """
    kfold = KFold(n_splits=n_splits)
    fold = 1
    for train_idx, test_idx in kfold.split(X):
        print(f'FOLD {fold}')
        X_train, y_train = X.iloc[train_idx, :], y[train_idx]
        X_test, y_test = X.iloc[test_idx, :], y[test_idx]

        model.fit(X_train, y_train)

        print_scores(model, X_test, y_test, **kwargs)
        print()
        fold += 1

In [8]:
print('RANDOM CLASSIFIER KFOLD SCORES')
print_kfold_scores(random_clf, X, y, 5)

RANDOM CLASSIFIER KFOLD SCORES
FOLD 1
accuracy - 0.1714
f1_micro - 0.1714
precision_micro - 0.1714
recall_micro - 0.1714

FOLD 2
accuracy - 0.3143
f1_micro - 0.3143
precision_micro - 0.3143
recall_micro - 0.3143

FOLD 3
accuracy - 0.1714
f1_micro - 0.1714
precision_micro - 0.1714
recall_micro - 0.1714

FOLD 4
accuracy - 0.3429
f1_micro - 0.3429
precision_micro - 0.3429
recall_micro - 0.3429

FOLD 5
accuracy - 0.3429
f1_micro - 0.3429
precision_micro - 0.3429
recall_micro - 0.3429



In [7]:
print('MAX PEAK CLASSIFIER KFOLD SCORES')
print_kfold_scores(max_peak_clf, X, y, 5)

MAX PEAK CLASSIFIER KFOLD SCORES
FOLD 1
accuracy - 0.6286
f1_micro - 0.6286
precision_micro - 0.6286
recall_micro - 0.6286

FOLD 2
accuracy - 0.8286
f1_micro - 0.8286
precision_micro - 0.8286
recall_micro - 0.8286

FOLD 3
accuracy - 0.9143
f1_micro - 0.9143
precision_micro - 0.9143
recall_micro - 0.9143

FOLD 4
accuracy - 0.9714
f1_micro - 0.9714
precision_micro - 0.9714
recall_micro - 0.9714

FOLD 5
accuracy - 0.9714
f1_micro - 0.9714
precision_micro - 0.9714
recall_micro - 0.9714



Random classifier ranges from 0.1429 - 0.2286
Max peak classifier ranges from 0.6286 - 0.9714

In [20]:
def calc_average_kfold_accuracy(model, n_splits=5):
    kfold = KFold(n_splits=5)
    fold = 1
    accuracies = []
    for _, test_idx in kfold.split(X):
        X_test, y_test = X.iloc[test_idx, :], y[test_idx]
        acc = accuracy_score(y_test, model.predict(X_test))
        accuracies.append(acc)
    return np.mean(accuracies)

In [21]:
calc_average_kfold_accuracy(random_clf)

0.26285714285714284

In [22]:
calc_average_kfold_accuracy(max_peak_clf)

0.8628571428571428

The average accuracies for each model are the same as if the model makes predictions on the whole dataset. This makes sense.

I did not split the dataset into train/val/test sets before creating these baseline models. For the `RandomClassifier` this is not an issue since the prediction is not based on the sample. However, for the `MaxPeakClassifier` it will have caused some data leakage - I used the whole population to create these rules and, if I just used a training set, I would not necessarily have come up with the same rules. However, I decided to do this for brevity and to get a better understanding of the dataset as a whole. So, the 86.29% accuracy result is probably an overexaggeration of the model's power.But this is a decent baseline from which to compare everything else to. 86% is very strong for a rule based approach and the best an ML model can hope to achieve is a 16.3% increase in performance. 

Yes there could be some data leakage in here but really only in the value for seawater we would choose and it's clear that the vast majority 90%+ of our seawater samples stay below 2.5. Moreover, the peak sizes were chosen to be wide enough to have room for all samples. So probably a bit of data leakage but for brevity we will work with it and can say that 86% is the minimum that our ML models need to achieve. 